## Reqs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !git clone https://github.com/NeuralClassifier/ReusabilityRate_ML
# !git checkout corr_analysis
%cd /content/ReusabilityRate_ML/DataSets

/content/ReusabilityRate_ML/DataSets


In [ ]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statistics
import tensorflow as tf

import warnings
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Read class and package metrics(x:inputs) and RR(y:label)

In [ ]:
cls_df = pd.read_excel('classes.xlsx')
pkg_df = pd.read_excel('packages.xlsx')
cls_df.drop(['Unnamed: 0'], axis=1, inplace=True)
pkg_df.drop(['Unnamed: 0'], axis=1, inplace=True)

## Class DNN

### Load data

In [ ]:
target_scaler = MinMaxScaler(feature_range=(-10, 10))
data = cls_df
df = data.iloc[:,3:-1]
data_columns = df.columns
X_np = np.array(df)
target_scaler.fit(X_np)
X = X_np
#X = target_scaler.transform(X_np)

Y = data.iloc[:,-1]
Y = Y.values.reshape(-1, 1)
#target_scaler = MinMaxScaler(feature_range=(-10, 10))
#target_scaler.fit(Y)
#Y = target_scaler.transform(Y)

### Training data preparation 

In [ ]:
#indexes = np.arange(len(X))
splitter = int(len(X) * .8)
X_train = X[:splitter]
Y_train = Y[:splitter]
X_val = X[splitter:]
Y_val = Y[splitter:]

### Train model

In [ ]:
##model

def my_model_fully_connected(loss_fn, ishape = 27, lr = 1e-2):
    model_input = tf.keras.layers.Input(shape = ishape)
    l2 = tf.keras.layers.Dense(20, activation = 'sigmoid')(model_input)
    l2_drop = tf.keras.layers.Dropout(0.05)(l2)
    l3 = tf.keras.layers.Dense(20, activation = 'sigmoid')(l2_drop) 
    l4 = tf.keras.layers.Dense(5, activation = 'sigmoid')(l3)
    out= tf.keras.layers.Dense(1, activation = 'relu')(l4)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ["accuracy"])
    return model

loss_fn = tf.keras.losses.MeanSquaredError()
model = my_model_fully_connected(loss_fn, ishape=len(X_train[0]), lr= 0.1)
print(model.summary())


In [ ]:
Y_train[0:20]

In [ ]:
# Call backs 
model_name_fcn = "First_draft.h5"
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20)

monitor = tf.keras.callbacks.ModelCheckpoint(model_name_fcn, monitor='val_loss',\
                                             verbose=0, save_best_only=True,\
                                             save_weights_only=True,\
                                             mode='min')
# Learning rate schedule
def scheduler(epoch, lr):
    if epoch%10 == 0:
        lr = lr/2
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose = 0)

In [ ]:
# Train
model.fit(X_train, Y_train, epochs = 100, batch_size=200, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule], validation_data= (X_val, Y_val))

Epoch 1/100
100/100 [==============================] - 1s 4ms/step - loss: -27.8684 - accuracy: 0.5083 - val_loss: -44.0212 - val_accuracy: 0.1972 - lr: 5.0000e-04
Epoch 2/100
100/100 [==============================] - 0s 3ms/step - loss: -49.6527 - accuracy: 0.4136 - val_loss: -44.1544 - val_accuracy: 0.4211 - lr: 5.0000e-04
Epoch 3/100
100/100 [==============================] - 0s 3ms/step - loss: -50.7730 - accuracy: 0.4968 - val_loss: -44.6069 - val_accuracy: 0.2632 - lr: 5.0000e-04
Epoch 4/100
100/100 [==============================] - 0s 3ms/step - loss: -52.7089 - accuracy: 0.4656 - val_loss: -45.0806 - val_accuracy: 0.3077 - lr: 5.0000e-04
Epoch 5/100
100/100 [==============================] - 0s 3ms/step - loss: -51.8382 - accuracy: 0.4790 - val_loss: -44.7404 - val_accuracy: 0.2680 - lr: 5.0000e-04
Epoch 6/100
100/100 [==============================] - 0s 3ms/step - loss: -55.1378 - accuracy: 0.3982 - val_loss: -44.2041 - val_accuracy: 0.2259 - lr: 5.0000e-04
Epoch 7/100
100/

In [ ]:
print(str(len(X_train[0])))
[model.predict(X_train[3].reshape([1,27])), Y_train[0]]

27


[array([[0.]], dtype=float32), array([6])]

## Package DNN

### Load data

In [ ]:
target_scaler = MinMaxScaler()
data = pkg_df
df = data.iloc[:,3:-1]
data_columns = df.columns
X_np = np.array(df)
target_scaler.fit(X_np)
X = target_scaler.transform(X_np)

Y = data.iloc[:,-1]
Y = Y.values.reshape(-1, 1)
target_scaler = MinMaxScaler()
target_scaler.fit(Y)
Y = target_scaler.transform(Y)


### Train model

In [ ]:
## Train model

In [ ]:
## Test model